In [1]:
import os
import cv2
import numpy as np
import torch

In [3]:
# Specify the folder containing the images and the output folder
input_folder = '../datasets/thermal/images/train2017/'
output_folder_t = '../datasets/t-yolov5/images'
output_folder_t2 = '../datasets/t2-yolov5/images'
image_files = os.listdir(input_folder)
    
# Sort the image files
#image_files.sort()

# Iterate over each image file
for i, image_file in enumerate(image_files):
    if image_file.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
        # Read the current image
        current_image_path = os.path.join(input_folder, image_file)
        current_image = cv2.resize(cv2.imread(current_image_path, cv2.IMREAD_GRAYSCALE),(640,640))
        # Get the file names of the previous and next images
        prev_image_file = image_files[i-1] if i > 0 else image_files[i]
        next_image_file = image_files[i+1] if i < len(image_files)-1 else image_files[i]
        # Read the previous and next images
        prev_image = cv2.resize((cv2.imread(os.path.join(input_folder, prev_image_file), cv2.IMREAD_GRAYSCALE) if prev_image_file else None),(640,640))
        next_image = cv2.resize((cv2.imread(os.path.join(input_folder, next_image_file), cv2.IMREAD_GRAYSCALE) if next_image_file else None),(640,640))
        
        first_channel = np.abs(prev_image-current_image)
        second_channel= np.abs(next_image-current_image)

        t_image = np.dstack([current_image,prev_image,next_image])
        t2_image = np.dstack([first_channel,second_channel,np.zeros_like(first_channel)])
        t2_image = np.concatenate([t_image,t2_image],axis=1)
        # Save the transformed image
        output_path_t = os.path.join(output_folder_t, image_file)
        output_path_t2 = os.path.join(output_folder_t2, image_file)
        cv2.imwrite(output_path_t, t_image)
        cv2.imwrite(output_path_t2,t2_image)



In [ ]:
!python train.py --img 640 --epochs 5 --data t-yolov5.yaml --weights yolov5x.pt --name t-yolov5

In [61]:
!python train.py --img 640 --epochs 5 --weights "" --data t2-yolov5.yaml --cfg yolov5x-t2.yaml --name t2-yolov5 --t2_yolov5 True

WARNING ⚠️ 'ultralytics.yolo.v8' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.models.yolo' instead.
WARNING ⚠️ 'ultralytics.yolo.utils' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.utils' instead.
Note this warning may be related to loading older models. You can update your model to current structure with:
    import torch
    ckpt = torch.load("model.pt")  # applies to both official and custom models
    torch.save(ckpt, "updated-model.pt")

train: weights=, cfg=yolov5x-t2.yaml, data=t2-yolov5.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=5, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=t2-yolov5, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=10

In [19]:
t_yolo = torch.load("/usr/src/app/runs/train/t-yolov52/weights/best.pt")

In [23]:
t_yolo.get('model').model[5].state_dict()

OrderedDict([('conv.weight',
              tensor([[[[ 4.80652e-03,  9.31549e-03,  7.16019e-03],
                        [-8.06332e-04,  1.72424e-02,  1.12534e-02],
                        [-9.14097e-04,  1.10626e-03,  5.50842e-03]],
              
                       [[-6.74438e-03, -4.56238e-03, -6.03485e-03],
                        [ 4.58002e-04,  1.10321e-02,  6.96182e-03],
                        [-1.38760e-04,  3.48473e-03,  6.54984e-03]],
              
                       [[ 5.64575e-03,  3.02505e-03,  3.52097e-03],
                        [ 1.36414e-02,  2.25525e-02,  1.71967e-02],
                        [ 9.09424e-03,  1.45493e-02,  1.14746e-02]],
              
                       ...,
              
                       [[-6.69861e-03, -6.77872e-03, -8.41522e-03],
                        [ 9.30023e-03,  1.84631e-02,  1.84326e-02],
                        [ 1.27640e-02,  1.76392e-02,  1.49689e-02]],
              
                       [[-8.89587e-03, -9.04083e